In [2]:
from utils.image_utils import plot_XY, gen_index_file
from unet.dataset import SegThorImagesDataset
import torch
from torch.utils.data import DataLoader
from unet.unet_model import UNet
from torch import optim

In [3]:
# if index file doesn't exist, generate and save 
index_file_train = gen_index_file()

Filename: data/train_patient_idx.csv already exists, skipping gen


In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. ")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU instead.")

GPU is not available, using CPU instead.


In [5]:
# define model
# define training loop 
train_dataset = SegThorImagesDataset() 
train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
# todo: split this into train/validate

model = UNet(n_channels=1, n_classes=5) # 0: no-classification 1: organ, 2: organ, 3: organ, 4: organ

criterion = torch.nn.CrossEntropyLoss()  # or any other applicable loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5  # Number of epochs
model.to(device)

model.train()
epoch_loss = 0.0

iter_dl = iter(train_dl)

In [10]:
# define model
# define training loop 
train_dataset = SegThorImagesDataset() # TODO: training img transforms
train_dl = DataLoader(train_dataset, batch_size=16, shuffle=True)
# todo: split this into train/validate

model = UNet(n_channels=1, n_classes=5) # 0: no-classification 1: organ, 2: organ, 3: organ, 4: organ
criterion = torch.nn.CrossEntropyLoss()  # or any other applicable loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5  # Number of epochs
model.to(device)


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_dl:
        inputs = inputs.to(device)
        outputs = model(inputs)
        outputs = outputs.cpu()
        #predictions = torch.argmax(outputs, dim=1).float()
        loss = criterion(outputs, targets.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss -= loss.item()
        inputs.cpu()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_dl)}')

model.cpu()
